# Analyzing 311 complaints per tract and merging data with Census analysis

This notebook first analyzes 311 data the following way: 
* for 2016 (so we can calculate per capita rates of complaints based on total tract population counts from the 2016 ACS)
* pare down to categories that show neighbors complaining about neighbors (combination of sociologist [Joscha Legewie's research](http://jlegewie.com/files/Legewie-Schaeffer-2016-ContestedBoundaries.pdf) and editorial decisions made by BuzzFeed News)
* assigns each row to a census tract based on that latitude and longitude of the 311 complaint
* does a count of the 311 complaints per tract and merges 311 data with geojson data of NYC into one geodataframe

Then the notebook adds the data from the previous notebook `02-gentrification_measure_and_race_analysis` to the analysis:
* joins and the 311 geodataframe with `gentrification.csv` which contains data about whether a tract was gentrified or not, as well as percentage point changes of ethnic groups between 2000 and 2016
* calculates rates of complaints per capita 
* calculates medians, means for gentrified tracts and all other tracts
* filters data for on-the-ground reporting (looking into two neighborhoods with high rates and numbers of complaints as well as a list of addresses associated with a high number of complaints for reporting purposes)


In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gp
from scipy.stats import ttest_ind

## Pare down 311 data
* Load all 311 complaints (bulk downloaded from [here](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9))
* cut off data by year (2016 only)

In [2]:
all_complaints = pd.read_csv(
    '../data/nyc/311_Service_Requests_from_2010_to_Present.csv',
    usecols = [
        'Created Date',
        'Complaint Type',
        'Descriptor',
        'Incident Address',
        'Resolution Description',
        'Latitude',
        'Longitude'
    ]
)

In [3]:
all_complaints["Created Date"] = pd.to_datetime(
    all_complaints["Created Date"],
    format = "%m/%d/%Y %H:%M:%S %p"   
)

In [4]:
print(len(all_complaints))
all_complaints.head()

17648719


,Created Date,Complaint Type,Descriptor,Incident Address,Resolution Description,Latitude,Longitude
0,2017-04-19 09:06:41,For Hire Vehicle Complaint,Driver Complaint,27 COLUMBUS AVENUE,The Taxi and Limousine Commission will contact...,40.770107,-73.984115
1,2017-04-19 05:04:48,Noise - Vehicle,Car/Truck Music,184 EAST FORDHAM ROAD,The Police Department responded to the complai...,40.862417,-73.897562
2,2017-04-19 03:25:15,Broken Muni Meter,Coin or Card Did Not Register,4202 8 AVENUE,General maintenance and cleaning is on a regul...,40.645513,-73.999153
3,2017-04-19 11:57:27,Broken Muni Meter,No Receipt,29-20B 23 AVENUE,General maintenance and cleaning is on a regul...,40.774940,-73.913702
4,2017-04-19 02:39:38,Overgrown Tree/Branches,Blocking Street,58-07 196 STREET,The Department of Parks and Recreation has det...,40.744141,-73.781149


In [5]:
all_complaints.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17648719 entries, 0 to 17648718
Data columns (total 7 columns):
Created Date              datetime64[ns]
Complaint Type            object
Descriptor                object
Incident Address          object
Resolution Description    object
Latitude                  float64
Longitude                 float64
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 942.5+ MB


#### Look into complaints type occurrences

In [6]:
print(all_complaints['Descriptor'].nunique())

1486


In [7]:
types_of_complaints = all_complaints['Descriptor'].value_counts()
types_of_complaints.head()

Loud Music/Party    1386763
HEAT                 871935
ENTIRE BUILDING      600405
Street Light Out     598961
No Access            520955
Name: Descriptor, dtype: int64

In [8]:
types_of_complaints.to_csv('../output/call-descriptor-counts.csv')

In [9]:
types_subtypes = (
    all_complaints.groupby([
        'Complaint Type',
        'Descriptor'
    ])
    .size()
    .to_frame("count")
)
types_subtypes.head()

count
Complaint Type Descriptor                      
AGENCY         HOUSING QUALITY STANDARDS      8
APPLIANCE      AIR-CONDITIONER              227
               ELECTRIC/GAS RANGE         32322
               ELECTRIC/GAS-RANGE         24795
               MICROWAVE                     33

In [10]:
types_subtypes.to_csv('../output/subtypes_complaints.csv')

## Pare down data

Pare down data based on:
* for 2016 (so we can calculate per capita rates of complaints based on total tract population counts from the 2016 ACS)
* pare down to categories that show neighbors complaining about neighbors (combination of sociologist [Joscha Legewie's research](http://jlegewie.com/files/Legewie-Schaeffer-2016-ContestedBoundaries.pdf) and editorial decisions made by BuzzFeed News)


In [11]:
complaints2016 = all_complaints[
    all_complaints['Created Date'].dt.year == 2016
]

In [12]:
print(len(complaints2016))
complaints2016.head()

2349293


,Created Date,Complaint Type,Descriptor,Incident Address,Resolution Description,Latitude,Longitude
4521,2016-12-12 01:46:03,General Construction/Plumbing,Building Shaking/Vibrating/Structural Stability,108 SOUTH STREET,The Department of Buildings investigated this ...,40.706836,-74.002121
27240,2016-08-25 12:00:00,ATF,NaN,1 WARDS ISLAND,NaN,40.786373,-73.931656
36815,2016-05-05 12:00:00,ATF,NaN,58-03 GRAND AVENUE,NaN,40.719675,-73.911168
43853,2016-09-15 12:00:00,ATF,NaN,43-01 BERRIAN BOULEVARD,NaN,40.778421,-73.894593
44725,2016-09-20 12:00:00,ATF,NaN,43-01 BERRIAN BOULEVARD,NaN,40.778421,-73.894593


## Run analysis to assign Census tract to pared down complaints
* load geodata
* turn lat long into a shapely Point object
* detect whether Point is inside a Census tract and assign the tract number to data


In [13]:
data_categories = {
    "Animal in a Park": [
        "Dog Off Leash"
    ],
    "Blocked Driveway": [
        "No Access",
        "Partial Access"
    ],
    "Building/Use": [
        "Illegal Conversion Of Residential Building/Space",            
    ],
    "Consumer Complaint": [
        "Cars Parked on Sidewalk/Street",
        "Cigarette Sale to Minor"
    ],
    "Drinking": [
        "In Public"
    ],
    "Drug Activity": [
        "Use Indoor",
        "Use Outside"
    ],
    "Noise": [
        "Noise, Barking Dog (NR5)",
        "Noise: Loud Music/Daytime (Mark Date And Time) (NN1)",
        "Noise: Loud Music/Nighttime(Mark Date And Time) (NP1)",
    ],
    "Noise - House of Worship": [
        "Banging/Pounding",
        "Loud Music/Party",
        "Loud Talking",
        "Loud Television"
    ],
    "Noise - Residential": [
        "Banging/Pounding",
        "Loud Music/Party",
        "Loud Talking",
        "Loud Television"
    ],
    "Noise - Street/Sidewalk": [
        "Loud Music/Party",
        "Loud Talking",
    ],
    "Non-Emergency Police Matter": [
        "Street Con Game"
    ],
    "Smoking": [
        "Smoking Violation"
    ],    
}

In [14]:
def is_category_of_interest(row):
    t = row["Complaint Type"]
    if t in data_categories.keys():
        if row["Descriptor"] in data_categories[t]:
            return True
    return False

In [15]:
complaints2016_filtered_cat = complaints2016[
    complaints2016.apply(is_category_of_interest, axis = 1)
]

In [16]:
complaints2016_filtered_cat_counts = (
    complaints2016_filtered_cat
    .groupby([
        "Complaint Type",
        "Descriptor"
    ])
    .size()
    .to_frame("count")
)

complaints2016_filtered_cat_counts

count
Complaint Type              Descriptor                                                
Animal in a Park            Dog Off Leash                                          771
Blocked Driveway            No Access                                            87734
                            Partial Access                                       31312
Building/Use                Illegal Conversion Of Residential Building/Space     18050
Consumer Complaint          Cars Parked on Sidewalk/Street                         578
                            Cigarette Sale to Minor                                382
Drinking                    In Public                                             1040
Drug Activity               Use Indoor                                             284
                            Use Outside                                             86
Noise                       Noise, Barking Dog (NR5)                              6731
                            Noise: Loud Music/Daytime (Mark Date And Time) ...       5
                            Noise: Loud Music/Nighttime(Mark Date And Time)...       6
Noise - House of Worship    Banging/Pounding                                        94
                            Loud Music/Party                                       600
                            Loud Talking                                           224
                            Loud Television                                          3
Noise - Residential         Banging/Pounding                                     56299
                            Loud Music/Party                                    145617
                            Loud Talking                                         16053
                            Loud Television                                       3937
Noise - Street/Sidewalk     Loud Music/Party                                     42319
                            Loud Talking                                         18880
Non-Emergency Police Matter Street Con Game                                         17
Smoking                     Smoking Violation                                     1621

In [17]:
complaints2016_filtered_cat_wlocations = (
    complaints2016_filtered_cat
    .dropna(subset = ['Latitude'])
    .copy()
)

len(complaints2016_filtered_cat_wlocations)

429655

Combine the latitude and longitude fields into a Point object


In [18]:
complaints2016_filtered_cat_wlocations['geometry'] = (
    complaints2016_filtered_cat_wlocations
    .apply(lambda z: Point(z["Longitude"], z["Latitude"]), axis = 1)
)

In [19]:
complaints2016_geodf = gp.GeoDataFrame(complaints2016_filtered_cat_wlocations)

In [20]:
complaints2016_geodf.crs

### Read in the polygon data
* read in the NY census tracts
* filter it down to NYC census tracts


In [21]:
ny_state_tracts = (
    gp.read_file(
        "../data/censusTracts/cb_2016_36_tract_500k/cb_2016_36_tract_500k.shp"
    )
    .drop(columns = [
        "TRACTCE",
        "AFFGEOID",
        "NAME",
        "ALAND",
        "AWATER",
        "LSAD"
    ])
)

ny_state_tracts.head()

,STATEFP,COUNTYFP,GEOID,geometry
0,36,051,36051031000,"POLYGON ((-77.841638 42.671857, -77.841531 42...."
1,36,053,36053030102,"POLYGON ((-75.69125 43.09777099999999, -75.686..."
2,36,055,36055000200,"POLYGON ((-77.63571399999999 43.17025, -77.632..."
3,36,055,36055001000,"POLYGON ((-77.592748 43.158266, -77.5916189999..."
4,36,055,36055002900,"POLYGON ((-77.59551499999999 43.152906, -77.59..."


In [22]:
NYC_COUNTIES = [
    "005", # Bronx
    "047", # Kings (Brooklyn)
    "061", # New York County (Manhattan)
    "081", # Queens
    "085", # Richmond (Staten Island)
]

In [23]:
nyc_census_tracts = ny_state_tracts[
    ny_state_tracts['COUNTYFP'].isin(NYC_COUNTIES)
]

In [24]:
len(nyc_census_tracts)

2166

In [25]:
nyc_census_tracts.crs

{'init': 'epsg:4269'}

In [26]:
complaints2016_geodf.crs = nyc_census_tracts.crs

### Merge geodataframes

In [27]:
merged_gdf = gp.sjoin(
    nyc_census_tracts,
    complaints2016_geodf,
    how = "inner"
)

In [28]:
print(len(merged_gdf))

429616


### Group and count complaints by county


In [29]:
total_complaints_by_tract = (
    merged_gdf
    .groupby([
        "GEOID",
    ])
    .size()
    .to_frame("complaints")
    .reset_index()
)

total_complaints_by_tract.head()

,GEOID,complaints
0,36005000100,1
1,36005000200,271
2,36005000400,237
3,36005001600,169
4,36005001900,149


In [30]:
unique_complaints_by_tract = (
    merged_gdf
    .assign(
        ymd = lambda x: x["Created Date"].dt.strftime("%Y-%m-%d")
    )
    .drop_duplicates(subset = [
        "GEOID",
        "ymd",
        "Incident Address",
        "Complaint Type",
        "Descriptor"
    ])
    .groupby([
        "GEOID",
    ])
    .size()
    .to_frame("unique_complaints")
    .reset_index()
)

unique_complaints_by_tract.head()

,GEOID,unique_complaints
0,36005000100,1
1,36005000200,241
2,36005000400,232
3,36005001600,149
4,36005001900,141


In [31]:
tract_counts_gdf = (
    nyc_census_tracts
    .merge(
        total_complaints_by_tract,
        on = "GEOID",
        how = "left"
    )
    .merge(
        unique_complaints_by_tract,
        on = "GEOID",
        how = "left"
    )
    .assign(
        complaints = lambda df: df["complaints"].fillna(0).astype(int),
        unique_complaints = lambda df: df["unique_complaints"].fillna(0).astype(int),
    )
)

tract_counts_gdf.head(10)

,STATEFP,COUNTYFP,GEOID,geometry,complaints,unique_complaints
0,36,061,36061001300,"POLYGON ((-74.015513 40.707707, -74.014697 40....",131,106
1,36,061,36061002202,"POLYGON ((-73.982964 40.722671, -73.9806069999...",167,152
2,36,061,36061003400,"POLYGON ((-73.984448 40.728781, -73.983543 40....",770,608
3,36,061,36061005501,"POLYGON ((-73.998955 40.728068, -73.9978859999...",45,40
4,36,061,36061006700,"POLYGON ((-74.006624 40.731669, -74.005706 40....",244,220
5,36,061,36061007800,"POLYGON ((-73.97763599999999 40.746392, -73.97...",173,159
6,36,061,36061008300,"POLYGON ((-74.007238 40.74346999999999, -74.00...",126,122
7,36,061,36061008800,"POLYGON ((-73.975835 40.748863, -73.9744489999...",58,53
8,36,061,36061009100,"POLYGON ((-73.998486 40.745294, -73.998009 40....",169,156
9,36,061,36061010400,"POLYGON ((-73.98042599999999 40.759899, -73.97...",71,56


In [32]:
len(tract_counts_gdf)

2166

In [33]:
complaint_per_tract = tract_counts_gdf[[
    'GEOID',
    'COUNTYFP',
    'geometry',
    'complaints',
    'unique_complaints'
]].sort_values("complaints", ascending = False)

complaint_per_tract.head(25)

,GEOID,COUNTYFP,geometry,complaints,unique_complaints
654,36047002901,047,"POLYGON ((-73.98038799999999 40.696507, -73.97...",4134,185
1197,36061022301,061,"POLYGON ((-73.9636574765412 40.8262983766213, ...",1909,1254
24,36061029100,061,"POLYGON ((-73.927269 40.865535, -73.925005 40....",1686,1184
1461,36005007000,005,"POLYGON ((-73.86872 40.829558, -73.867762 40.8...",1579,1057
810,36061027900,061,"POLYGON ((-73.93462699999999 40.854501, -73.93...",1558,1216
1981,36061028500,061,"POLYGON ((-73.93017399999999 40.861649, -73.93...",1525,1199
21,36061024500,061,"POLYGON ((-73.94429699999999 40.837078, -73.94...",1279,1127
1198,36061022500,061,"POLYGON ((-73.963182 40.8269, -73.962349903972...",1273,1062
992,36005018900,005,"POLYGON ((-73.933144 40.834815, -73.9303549999...",1142,938
811,36061028300,061,POLYGON ((-73.93181799999999 40.85827099999999...,1104,875


## Do analysis with census population data
* load census tract-level data
* merge counts based on `GEOID` column
* calculate per capita complaint rates

In [34]:
census_population_2016 = pd.read_csv(
    '../output/2016_census_data.csv', 
    dtype = {'geoid': str },
    usecols = [
        'geoid',
        'name',
        'total_population'
    ]
).rename(
    columns = {'geoid': 'GEOID'}
)

census_population_2016.sort_values(
    by = 'total_population',
    ascending = False
).head(5)

,GEOID,name,total_population
1777,36005046201,"Census Tract 462.01, Bronx County, New York",29256
2950,36061004400,"Census Tract 44, New York County, New York",16039
3160,36061024500,"Census Tract 245, New York County, New York",15642
3063,36061015400,"Census Tract 154, New York County, New York",15502
3042,36061013600,"Census Tract 136, New York County, New York",15234


In [35]:
print(len(complaint_per_tract))
print(len(census_population_2016))

merged_complaints_per_tract = pd.merge(
    complaint_per_tract,
    census_population_2016,
    on = 'GEOID',
    how = 'inner'
)

print(len(merged_complaints_per_tract))
merged_complaints_per_tract.head(20)

2166
4700
2166


,GEOID,COUNTYFP,geometry,complaints,unique_complaints,name,total_population
0,36047002901,047,"POLYGON ((-73.98038799999999 40.696507, -73.97...",4134,185,"Census Tract 29.01, Kings County, New York",4074
1,36061022301,061,"POLYGON ((-73.9636574765412 40.8262983766213, ...",1909,1254,"Census Tract 223.01, New York County, New York",8022
2,36061029100,061,"POLYGON ((-73.927269 40.865535, -73.925005 40....",1686,1184,"Census Tract 291, New York County, New York",12188
3,36005007000,005,"POLYGON ((-73.86872 40.829558, -73.867762 40.8...",1579,1057,"Census Tract 70, Bronx County, New York",4889
4,36061027900,061,"POLYGON ((-73.93462699999999 40.854501, -73.93...",1558,1216,"Census Tract 279, New York County, New York",12413
5,36061028500,061,"POLYGON ((-73.93017399999999 40.861649, -73.93...",1525,1199,"Census Tract 285, New York County, New York",7050
6,36061024500,061,"POLYGON ((-73.94429699999999 40.837078, -73.94...",1279,1127,"Census Tract 245, New York County, New York",15642
7,36061022500,061,"POLYGON ((-73.963182 40.8269, -73.962349903972...",1273,1062,"Census Tract 225, New York County, New York",10631
8,36005018900,005,"POLYGON ((-73.933144 40.834815, -73.9303549999...",1142,938,"Census Tract 189, Bronx County, New York",8200
9,36061028300,061,POLYGON ((-73.93181799999999 40.85827099999999...,1104,875,"Census Tract 283, New York County, New York",7653


In [36]:
merged_complaints_per_tract_capita = merged_complaints_per_tract[
    merged_complaints_per_tract['total_population'] > 499
].copy()

merged_complaints_per_tract_capita['complaints_per_capita'] = (
    merged_complaints_per_tract['complaints'] / 
    merged_complaints_per_tract['total_population']
)

merged_complaints_per_tract_capita['unique_complaints_per_capita'] = (
    merged_complaints_per_tract['unique_complaints'] / 
    merged_complaints_per_tract['total_population']
)

print(len(merged_complaints_per_tract))
print(len(merged_complaints_per_tract_capita))

2166
2099


In [37]:
merged_complaints_per_tract_capita.sort_values(
    by = 'unique_complaints_per_capita',
    ascending = False
).head(5)

,GEOID,COUNTYFP,geometry,complaints,unique_complaints,name,total_population,complaints_per_capita,unique_complaints_per_capita
72,36081116700,081,"POLYGON ((-73.81638699999999 40.76316, -73.812...",582,428,"Census Tract 1167, Queens County, New York",1848,0.314935,0.231602
3,36005007000,005,"POLYGON ((-73.86872 40.829558, -73.867762 40.8...",1579,1057,"Census Tract 70, Bronx County, New York",4889,0.322970,0.216200
13,36061028700,061,"POLYGON ((-73.9381535187682 40.8745831121645, ...",1006,812,"Census Tract 287, New York County, New York",3960,0.254040,0.205051
605,36061011900,061,"POLYGON ((-73.98929699999999 40.757907, -73.98...",242,202,"Census Tract 119, New York County, New York",996,0.242972,0.202811
840,36081005500,081,"POLYGON ((-73.93029899999999 40.754548, -73.92...",192,168,"Census Tract 55, Queens County, New York",933,0.205788,0.180064


In [38]:
merged_complaints_per_tract_capita.describe()

,complaints,unique_complaints,total_population,complaints_per_capita,unique_complaints_per_capita
count,2099.000000,2099.000000,2099.000000,2099.000000,2099.000000
mean,203.204859,174.934731,4030.021439,0.051683,0.044807
std,196.139814,141.152376,2174.927729,0.039581,0.026707
min,0.000000,0.000000,507.000000,0.000000,0.000000
25%,89.000000,82.000000,2480.000000,0.028564,0.025867
50%,156.000000,139.000000,3638.000000,0.044475,0.040216
75%,262.000000,228.000000,5092.000000,0.067048,0.059594
max,4134.000000,1254.000000,29256.000000,1.014728,0.231602


In [39]:
gentrification_data = pd.read_csv(
    '../output/gentrification.csv',
     dtype = {
         'GEOID': str,
     },
     usecols = [
        'GEOID', 
        'gentrified',
        'gentrified',
        'low_population',
        'pct_white_alone_change',
        'eligible_for_gentrification',
        'eligible_not_gentrified_highpop',
        'not_eligible_highpop',
        'pct_white_alone_2016',
        'pct_black_alone_change',
        'pct_black_alone_2016',
        'pct_native_alone_change',
        'pct_native_alone_2016',
        'pct_asian_alone_change',
        'pct_asian_alone_2016',
        'pct_hispanic_or_latino_alone_change',
        'pct_hispanic_or_latino_alone_2016',
        'pct_native_hawaiian_pacific_islander_change',
        'pct_native_hawaiian_pacific_islander_2016',
     ]
)
gentrification_data.head()

,GEOID,gentrified,pct_white_alone_change,pct_white_alone_2016,pct_black_alone_change,pct_black_alone_2016,pct_native_alone_change,pct_native_alone_2016,pct_asian_alone_change,pct_asian_alone_2016,pct_hispanic_or_latino_alone_change,pct_hispanic_or_latino_alone_2016,pct_native_hawaiian_pacific_islander_change,pct_native_hawaiian_pacific_islander_2016,low_population,eligible_for_gentrification,eligible_not_gentrified_highpop,not_eligible_highpop
0,36005000100,False,-1.739495,6.797281,-0.695456,59.922698,0.093556,0.186592,0.352210,1.799280,2.914193,29.854725,0.0,0.0,False,False,False,True
1,36005000200,False,-4.124536,1.771091,-4.672665,18.091792,-0.073318,0.000000,-0.229119,2.799467,5.450804,74.957151,0.0,0.0,False,True,True,False
2,36005000400,False,-1.186907,4.063545,7.463280,29.732441,-0.121073,0.317726,-1.143472,0.568562,-3.362385,64.682274,0.0,0.0,False,True,True,False
3,36005001600,False,-2.428943,2.344782,-1.702927,29.326288,-0.064559,0.000000,-0.144612,0.000000,4.668782,66.727213,0.0,0.0,False,False,False,True
4,36005001900,True,1.525744,7.270694,2.890058,32.624907,-0.232611,0.000000,-0.387686,1.864280,-8.283760,54.026846,0.0,0.0,False,True,False,False


In [40]:
merged_complaints_final = pd.merge(
    merged_complaints_per_tract_capita,
    gentrification_data,
    on = 'GEOID',
    how = 'left'

)
print(len(merged_complaints_final))
merged_complaints_final.head(5)

2099


,GEOID,COUNTYFP,geometry,complaints,unique_complaints,name,total_population,complaints_per_capita,unique_complaints_per_capita,gentrified,...,pct_asian_alone_change,pct_asian_alone_2016,pct_hispanic_or_latino_alone_change,pct_hispanic_or_latino_alone_2016,pct_native_hawaiian_pacific_islander_change,pct_native_hawaiian_pacific_islander_2016,low_population,eligible_for_gentrification,eligible_not_gentrified_highpop,not_eligible_highpop
0,36047002901,047,"POLYGON ((-73.98038799999999 40.696507, -73.97...",4134,185,"Census Tract 29.01, Kings County, New York",4074,1.014728,0.045410,False,...,0.087726,19.759450,-0.528148,29.774178,-0.139535,0.0,False,False,False,True
1,36061022301,061,"POLYGON ((-73.9636574765412 40.8262983766213, ...",1909,1254,"Census Tract 223.01, New York County, New York",8022,0.237971,0.156320,True,...,0.083391,2.729993,-15.680532,74.295687,0.000000,0.0,False,True,False,False
2,36061029100,061,"POLYGON ((-73.927269 40.865535, -73.925005 40....",1686,1184,"Census Tract 291, New York County, New York",12188,0.138333,0.097145,False,...,0.020970,1.780440,-2.982447,87.906137,0.000000,0.0,False,True,True,False
3,36005007000,005,"POLYGON ((-73.86872 40.829558, -73.867762 40.8...",1579,1057,"Census Tract 70, Bronx County, New York",4889,0.322970,0.216200,False,...,-1.265286,6.708938,2.215000,63.080385,0.000000,0.0,False,False,False,True
4,36061027900,061,"POLYGON ((-73.93462699999999 40.854501, -73.93...",1558,1216,"Census Tract 279, New York County, New York",12413,0.125514,0.097962,False,...,-0.075555,2.199307,-1.815359,78.514461,-0.017001,0.0,False,True,True,False


In [41]:
(
    merged_complaints_final
    .replace(True, "True")
    .replace(False, "False")
    .to_file(
        '../output/merged_complaints_final.json',
        driver = "GeoJSON"
    )
)

### Finding neighborhood and adresses for on-the-ground reporting

This part of the script is for exploratory purposes, helping the reporter find gentrified neighborhoods with high levels of 311 complaints and interview people for the story.

Print a list of tracts with high per capita complaint rates:

In [42]:
gentrified_tracts = merged_complaints_final[
    merged_complaints_final['gentrified'] == True
].sort_values(
    by = 'unique_complaints_per_capita', 
    ascending = False
)
print(len(gentrified_tracts))
gentrified_tracts.head(5)

125


,GEOID,COUNTYFP,geometry,complaints,unique_complaints,name,total_population,complaints_per_capita,unique_complaints_per_capita,gentrified,...,pct_asian_alone_change,pct_asian_alone_2016,pct_hispanic_or_latino_alone_change,pct_hispanic_or_latino_alone_2016,pct_native_hawaiian_pacific_islander_change,pct_native_hawaiian_pacific_islander_2016,low_population,eligible_for_gentrification,eligible_not_gentrified_highpop,not_eligible_highpop
13,36061028700,061,"POLYGON ((-73.9381535187682 40.8745831121645, ...",1006,812,"Census Tract 287, New York County, New York",3960,0.254040,0.205051,True,...,0.364293,2.626263,-12.529784,58.989899,0.0,0.0,False,True,False,False
838,36081005500,081,"POLYGON ((-73.93029899999999 40.754548, -73.92...",192,168,"Census Tract 55, Queens County, New York",933,0.205788,0.180064,True,...,5.412912,23.151125,-17.527994,37.406217,0.0,0.0,False,True,False,False
21,36005006300,005,"POLYGON ((-73.933059 40.828156, -73.9311429999...",873,697,"Census Tract 63, Bronx County, New York",4291,0.203449,0.162433,True,...,1.792675,4.054999,6.842414,66.604521,0.0,0.0,False,True,False,False
95,36047048500,047,"POLYGON ((-73.94068799999999 40.711962, -73.93...",533,450,"Census Tract 485, Kings County, New York",2854,0.186755,0.157673,True,...,-1.428760,4.134548,-37.082269,38.367204,0.0,0.0,False,True,False,False
1,36061022301,061,"POLYGON ((-73.9636574765412 40.8262983766213, ...",1909,1254,"Census Tract 223.01, New York County, New York",8022,0.237971,0.156320,True,...,0.083391,2.729993,-15.680532,74.295687,0.0,0.0,False,True,False,False


### Stats for `gentrified` tracts

In [43]:
(
    merged_complaints_final
    .groupby([
        "low_population",
        "eligible_for_gentrification",
        "gentrified"
    ])
    [[
        "complaints_per_capita",
        "unique_complaints_per_capita",
        "pct_white_alone_change",
        "pct_black_alone_change",
        "pct_asian_alone_change",
        "pct_hispanic_or_latino_alone_change"
    ]]
    .describe()
    .round(3)
    .T
)

low_population                                False                     True 
eligible_for_gentrification                   False    True             False
gentrified                                    False    False    True    False
complaints_per_capita               count  1635.000  330.000  125.000   9.000
                                    mean      0.049    0.058    0.067   0.049
                                    std       0.040    0.031    0.043   0.042
                                    min       0.000    0.001    0.001   0.002
                                    25%       0.026    0.036    0.041   0.025
                                    50%       0.042    0.053    0.060   0.033
                                    75%       0.063    0.075    0.084   0.095
                                    max       1.015    0.216    0.254   0.117
unique_complaints_per_capita        count  1635.000  330.000  125.000   9.000
                                    mean      0.043    0.050    0.059   0.040
                                    std       0.026    0.024    0.035   0.031
                                    min       0.000    0.001    0.001   0.002
                                    25%       0.024    0.033    0.037   0.024
                                    50%       0.038    0.046    0.055   0.030
                                    75%       0.056    0.066    0.073   0.067
                                    max       0.232    0.170    0.205   0.091
pct_white_alone_change              count  1634.000  330.000  125.000   9.000
                                    mean     -4.254   -1.326    7.376   3.393
                                    std      12.036    7.667   11.948  31.186
                                    min     -44.646  -25.168  -19.639 -34.557
                                    25%     -11.197   -3.403   -0.984 -18.844
                                    50%      -2.985   -0.432    5.967  -6.104
                                    75%       1.199    1.051   15.107  35.960
                                    max      67.545   78.368   44.824  47.047
pct_black_alone_change              count  1634.000  330.000  125.000   9.000
                                    mean     -2.814   -4.770   -7.041 -12.034
                                    std       9.026    9.337   11.669  18.162
                                    min     -56.377  -45.662  -40.298 -44.690
                                    25%      -5.006   -9.492  -11.388 -22.542
                                    50%      -0.699   -3.105   -2.658  -8.504
                                    75%       0.734    0.802   -0.175  -3.570
                                    max      38.281   18.621   16.227  13.093
pct_asian_alone_change              count  1634.000  330.000  125.000   9.000
                                    mean     -0.341    0.132    0.280  -4.470
                                    std       2.672    2.750    2.225   4.234
                                    min     -21.237   -6.914   -6.971 -10.698
                                    25%      -0.763   -0.297   -0.390  -7.316
                                    50%      -0.091   -0.050   -0.049  -4.214
                                    75%       0.176    0.028    0.109  -0.538
                                    max      36.459   29.968   15.127   0.000
pct_hispanic_or_latino_alone_change count  1634.000  330.000  125.000   9.000
                                    mean      2.189    2.989   -3.392   0.419
                                    std       8.754    9.508    9.945  20.275
                                    min     -69.310  -69.299  -37.082 -31.595
                                    25%      -1.882   -2.240   -8.284 -16.385
                                    50%       2.111    3.210   -1.770  -0.911
                                    75%       6.631    9.176    3.283   9.636
                                    max      35.911   26.688   21.484  33.978

In [44]:
print(merged_complaints_final['pct_native_hawaiian_pacific_islander_2016'].mean())
print(merged_complaints_final['pct_native_alone_2016'].mean())


0.034367388880571774
0.3732744413667482


## Test for significance

In [45]:
ttest_ind(
    (
        merged_complaints_final
        .loc[lambda x: x["gentrified"] == True]
        ["unique_complaints_per_capita"]
    ),
    (
        merged_complaints_final
        .loc[lambda x: x["eligible_for_gentrification"] == True]
        .loc[lambda x: x["gentrified"] == False]
        ["unique_complaints_per_capita"]
    ),
)

Ttest_indResult(statistic=2.9571823938232464, pvalue=0.003266934618960035)

In [46]:
ttest_ind(
    (
        merged_complaints_final
        .loc[lambda x: x["gentrified"] == True]
        ["complaints_per_capita"]
    ),
    (
        merged_complaints_final
        .loc[lambda x: x["eligible_for_gentrification"] == True]
        .loc[lambda x: x["gentrified"] == False]
        ["complaints_per_capita"]
    ),
)

Ttest_indResult(statistic=2.461862997501917, pvalue=0.014193227621528443)

# Get complaints for specific Census tracts

Information for selected Census tracts (one in Brooklyn and one in Manhattan) that were gentrified and had a high rate of complaints: 

In [47]:
print(len(merged_gdf[merged_gdf['GEOID'] == '36047042900']))

bushwick_block = merged_gdf[
    merged_gdf['GEOID'] == '36047042900'
][[
    'GEOID', 
    'Created Date', 
    'Complaint Type', 
    'Descriptor',
    'Incident Address',
    'Latitude',
    'Longitude',
]]

(
    bushwick_block
    .groupby([
        'Incident Address',
        'Complaint Type',
        'Descriptor'
    ])
    .size()
    .sort_values(ascending = False)
    .to_frame("complaints")
    .reset_index()
    .head(20)
)

714


,Incident Address,Complaint Type,Descriptor,complaints
0,316 SUYDAM STREET,Noise - Residential,Loud Music/Party,94
1,1125 WILLOUGHBY AVENUE,Noise - Residential,Loud Music/Party,42
2,358 KNICKERBOCKER AVENUE,Noise - Residential,Banging/Pounding,24
3,175 STOCKHOLM STREET,Noise - Residential,Loud Music/Party,22
4,255 STOCKHOLM STREET,Noise - Residential,Loud Music/Party,14
5,263 STOCKHOLM STREET,Noise - Residential,Loud Music/Party,13
6,1433 DEKALB AVENUE,Noise - Residential,Loud Music/Party,12
7,DEKALB AVENUE,Noise - Street/Sidewalk,Loud Music/Party,11
8,847 HART STREET,Noise - Street/Sidewalk,Loud Music/Party,10
9,839 HART STREET,Noise - Residential,Loud Music/Party,10


In [48]:
print(len(merged_gdf[merged_gdf['GEOID'] == '36061022301']))

manhattan = merged_gdf[
    merged_gdf['GEOID'] == '36061022301'
][[
    'GEOID', 
    'Created Date', 
    'Complaint Type', 
    'Descriptor',
    'Incident Address',
    'Latitude',
    'Longitude',
]]

(
    manhattan
    .groupby([
        'Incident Address',
        'Complaint Type',
        'Descriptor'
    ])
    .size()
    .sort_values(ascending = False)
    .to_frame("complaints")
    .reset_index()
    .head(20)
)

1909


,Incident Address,Complaint Type,Descriptor,complaints
0,615 WEST 136 STREET,Noise - Residential,Loud Talking,169
1,618 WEST 136 STREET,Noise - Residential,Loud Talking,163
2,622 WEST 136 STREET,Noise - Residential,Loud Talking,130
3,615 WEST 136 STREET,Noise - Street/Sidewalk,Loud Talking,83
4,615 WEST 136 STREET,Noise - Residential,Loud Music/Party,79
5,618 WEST 136 STREET,Noise - Residential,Loud Music/Party,49
6,619 WEST 136 STREET,Noise - Residential,Loud Talking,48
7,522 WEST 136 STREET,Noise - Street/Sidewalk,Loud Music/Party,47
8,601 WEST 135 STREET,Noise - Residential,Loud Music/Party,44
9,615 WEST 136 STREET,Noise - Street/Sidewalk,Loud Music/Party,42


Search for complaints from addresses on the same block: `WEST 136 STREET` between Riverside Drive and Broadway:
* filter down data to desired periods: 2012 through 2017
* filter down to desired categories
* turn lat longs into points in a geometry column
* turn it into a geodf 
* filter down tract to just the one in harlem
* merge them

In [49]:
complaints_2012_2017 = all_complaints[
    (all_complaints['Created Date'] >= pd.to_datetime('2012-01-01'))&
    (all_complaints['Created Date'] < pd.to_datetime('2018-01-01'))
]
print(len(all_complaints))
print(len(complaints_2012_2017))

17648719
12718123


In [50]:
complaints_2012_2017_filtered = complaints_2012_2017[
    complaints_2012_2017.apply(is_category_of_interest, axis = 1)
].copy()

In [51]:
complaints_2012_2017_filtered['geometry'] = (
    complaints_2012_2017_filtered
    .apply(lambda z: Point(z["Longitude"], z["Latitude"]), axis = 1)
)

In [52]:
complaints_2012_2017_filtered = gp.GeoDataFrame(complaints2016_filtered_cat_wlocations)

In [53]:
complaints2016_geodf.crs

{'init': 'epsg:4269'}

In [54]:
harlem_tract = nyc_census_tracts[nyc_census_tracts['GEOID'] ==  '36061022301']


In [55]:
merged_harlem_complaints_2012_2017_gdf = gp.sjoin(
    harlem_tract,
    complaints_2012_2017_filtered,
    how = "inner"
)

In [56]:
len(merged_harlem_complaints_2012_2017_gdf)

1909

In [57]:
merged_harlem_complaints_2012_2017_gdf['Incident Address'].value_counts().to_csv('../output/harlem_unique_addresses.csv')

In [58]:
address_complaints = all_complaints[
    all_complaints['Incident Address'].isin([
        "600 WEST 136 STREET",
        "601 WEST 136 STREET",
        "602 WEST 136 STREET",
        "603 WEST 136 STREET",
        "604 WEST 136 STREET",
        "605 WEST 136 STREET",
        "606 WEST 136 STREET",
        "607 WEST 136 STREET",
        "608 WEST 136 STREET",
        "609 WEST 136 STREET",
        "610 WEST 136 STREET",
        "611 WEST 136 STREET",
        "612 WEST 136 STREET",
        "613 WEST 136 STREET",
        "614 WEST 136 STREET",
        "615 WEST 136 STREET",
        "616 WEST 136 STREET",
        "617 WEST 136 STREET",
        "618 WEST 136 STREET",
        "619 WEST 136 STREET",
        "620 WEST 136 STREET",
        "621 WEST 136 STREET",
        "622 WEST 136 STREET",
        "623 WEST 136 STREET",
        "624 WEST 136 STREET",
        "625 WEST 136 STREET",
        "626 WEST 136 STREET",
        "627 WEST 136 STREET",
        "628 WEST 136 STREET",
        "629 WEST 136 STREET",
        "630 WEST 136 STREET",
        "631 WEST 136 STREET",
        "632 WEST 136 STREET",
        "633 WEST 136 STREET",
        "634 WEST 136 STREET",
        "635 WEST 136 STREET",
        "636 WEST 136 STREET",
        '615 WEST 136 STREET',
        '618 WEST 136 STREET',
        '622 WEST 136 STREET',
        '619 WEST 136 STREET',
        '623 WEST 136 STREET',
        '611 WEST 136 STREET',
        '636 WEST 136 STREET',
        '621 WEST 136 STREET',
        '612 WEST 136 STREET',
        '607 WEST 136 STREET',
        '616 WEST 136 STREET',
        '624 WEST 136 STREET',
        '625 WEST 136 STREET',
        '626 WEST 136 STREET',
        '614 WEST 136 STREET',
        '600 WEST 136 STREET',
        '613 WEST 136 STREET',
        '620 WEST 136 STREET',
        '630 WEST 136 STREET',
        '623 WEST  136 STREET',
        '636 WEST  136 STREET',
        '622 WEST  136 STREET',
        '627 WEST 136 STREET',
        '634 WEST 136 STREET',
    ])
]

len(address_complaints)

4011

In [59]:
address_complaints_filtered = address_complaints[
    address_complaints.apply(is_category_of_interest, axis = 1)
].sort_values(
    by = 'Created Date'
).assign(
    date_only = lambda x: pd.to_datetime(x["Created Date"].dt.strftime("%Y-%m-%d"))
)

address_complaints_filtered.to_csv(
    '../output/all_complaints_block.csv',
    index = False
)
print(len(address_complaints_filtered))
address_complaints_filtered.head(20)

3225


,Created Date,Complaint Type,Descriptor,Incident Address,Resolution Description,Latitude,Longitude,date_only
7364462,2010-02-07 05:22:31,Noise - Residential,Loud Music/Party,636 WEST 136 STREET,NaN,40.821160,-73.955527,2010-02-07
7638111,2010-04-01 11:13:49,Noise - Residential,Loud Music/Party,600 WEST 136 STREET,NaN,40.820935,-73.954996,2010-04-01
7647239,2010-04-03 03:50:07,Noise - Street/Sidewalk,Loud Music/Party,600 WEST 136 STREET,NaN,40.820935,-73.954996,2010-04-03
7649234,2010-04-04 02:42:18,Noise - Residential,Loud Music/Party,615 WEST 136 STREET,NaN,40.821036,-73.955191,2010-04-04
7650190,2010-04-04 03:33:49,Noise - Residential,Loud Music/Party,615 WEST 136 STREET,NaN,40.821036,-73.955191,2010-04-04
7739850,2010-04-24 02:23:28,Noise - Residential,Loud Music/Party,636 WEST 136 STREET,NaN,40.821160,-73.955527,2010-04-24
7778412,2010-05-01 11:10:02,Noise - Residential,Loud Music/Party,619 WEST 136 STREET,NaN,40.821061,-73.955249,2010-05-01
7800070,2010-05-06 11:34:59,Noise - Residential,Loud Music/Party,622 WEST 136 STREET,NaN,40.821072,-73.955321,2010-05-06
7810396,2010-05-09 01:21:34,Noise - Residential,Loud Music/Party,619 WEST 136 STREET,NaN,40.821061,-73.955249,2010-05-09
7849953,2010-05-17 11:17:08,Noise - Street/Sidewalk,Loud Talking,615 WEST 136 STREET,NaN,40.821036,-73.955191,2010-05-17


In [60]:
address_complaints_filtered.dtypes

Created Date              datetime64[ns]
Complaint Type                    object
Descriptor                        object
Incident Address                  object
Resolution Description            object
Latitude                         float64
Longitude                        float64
date_only                 datetime64[ns]
dtype: object

In [61]:
daily_address_complaints_filtered = (
    address_complaints_filtered
    .groupby("date_only")
    .size()
    .to_frame("count")
)

daily_address_complaints_filtered.to_csv(
    '../output/all_complaints_block_for_viz.csv'
)

look for disctinct resolution descriptions

In [62]:
address_complaints_filtered_2012_2017 = address_complaints_filtered[
            (address_complaints_filtered['date_only'] >= pd.to_datetime('2012-01-01')) &
            (address_complaints_filtered['date_only'] <= pd.to_datetime('2017-12-31'))
          ]
address_complaints_filtered_2012_2017['Resolution Description'
           ].value_counts(
           ).reset_index(
           ).to_csv('../output/distinct_resolutions2012_2017.csv', index=False)

see if there are `null` values for resolution description between 2012 and 2017:

In [63]:
len(address_complaints_filtered_2012_2017[address_complaints_filtered_2012_2017['Resolution Description'].isna() == True]) 

0

Find some overview numbers for the written story about number of complaints on the block

In [64]:
print("highest number of complaints per day:", daily_address_complaints_filtered["count"].max())
print("number of complaints 2012-2014:", 
    address_complaints_filtered[
        (address_complaints_filtered['Created Date'] >='2012-01-01') &
        (address_complaints_filtered['Created Date'] < '2015-01-01')
    ].pipe(len)
)

print("number of complaints 2015-2017:", 
    address_complaints_filtered[
        (address_complaints_filtered['Created Date'] >='2015-01-01') &
        (address_complaints_filtered['Created Date'] < '2018-01-01')
    ].pipe(len)
)

highest number of complaints per day: 100
number of complaints 2012-2014: 131
number of complaints 2015-2017: 2993


Find complaints that police "responded", according to 311 status

In [65]:
complaints_harlem_since2015 = address_complaints_filtered[
    (address_complaints_filtered['Created Date'] >='2015-01-01') &
    (address_complaints_filtered['Created Date'] < '2018-01-01')
]

complaints_harlem_responses = (
    complaints_harlem_since2015[
        (complaints_harlem_since2015['Resolution Description'].str.contains('responded')) |
        (complaints_harlem_since2015['Resolution Description'].str.contains('summons'))
    ].groupby([
        "date_only",
        'Resolution Description'
    ])
    .size()
    .to_frame("count")
    .reset_index()
)

complaints_harlem_responses.to_csv('../output/dated_responses_to_complaints.csv',index=False)
print(len(complaints_harlem_responses))
complaints_harlem_responses.head(20)

855


,date_only,Resolution Description,count
0,2015-01-02,The Police Department responded to the complai...,2
1,2015-01-03,The Police Department responded to the complai...,1
2,2015-01-04,The Police Department responded to the complai...,1
3,2015-01-05,The Police Department responded to the complai...,1
4,2015-01-10,The Police Department responded to the complai...,1
5,2015-01-11,The Police Department responded to the complai...,1
6,2015-01-14,The Police Department responded to the complai...,2
7,2015-01-19,The Police Department responded to the complai...,1
8,2015-01-20,The Police Department responded to the complai...,1
9,2015-01-21,The Police Department responded to the complai...,1


Distinct number of dates when police responded to the complaint. (Over the years, an increasing proportion of 311 complaints' "Resolution Description"s indicate that police responded to the call — up from 13% of complaints in 2011 to 28% in 2017.)

In [66]:
complaints_harlem_responses['date_only'].nunique()

623

In [67]:
complaints_harlem_since2012_2014 =  address_complaints_filtered[
    (address_complaints_filtered['Created Date'] >='2012-01-01') &
    (address_complaints_filtered['Created Date'] < '2015-01-01')
]

complaints_harlem_2012_2014_responses = (
    complaints_harlem_since2012_2014[
        complaints_harlem_since2012_2014['Resolution Description'].str.contains('responded')|
        complaints_harlem_since2012_2014['Resolution Description'].str.contains('summons')
    ].groupby([
        'date_only', 
        'Resolution Description'
    ])
    .size()
    .to_frame("count")
    .reset_index()
)

complaints_harlem_2012_2014_responses['date_only'].nunique()

103

### Data work for graphics
export data for 3D data visualizations on maps:
* on a block-level

In [68]:
block_level = (
    address_complaints_filtered[
        (address_complaints_filtered['date_only'] >= '2015-01-01') &
        (address_complaints_filtered['date_only'] < '2018-01-01')
    ].groupby([
        'Incident Address',
        'Latitude',
        'Longitude'
    ])
    .size()
    .sort_values(ascending = False)
    .to_frame("counts")
    .reset_index()
)

block_level.to_csv('../output/block_level_dataviz.csv', index=False)

block_level.head()

,Incident Address,Latitude,Longitude,counts
0,615 WEST 136 STREET,40.821036,-73.955191,1118
1,622 WEST 136 STREET,40.821072,-73.955321,725
2,618 WEST 136 STREET,40.821047,-73.955260,505
3,619 WEST 136 STREET,40.821061,-73.955249,341
4,611 WEST 136 STREET,40.821012,-73.955134,70


---

---

---